<h2>Knn Regression</H2>

In [2]:
import numpy as np
import pandas as pd
import math as ma
import matplotlib.pyplot as plt

<h3>Defining helper functions</h3>

In [3]:
def get_csv(path: str) -> pd.DataFrame:
    
    df = pd.read_csv(path, parse_dates = ['Date'])\
        .drop(columns = ['Currency', '24h Open (USD)'])\
        .rename(columns = {'Date': 'date', 'Closing Price (USD)': 'price', '24h High (USD)': 'high', '24h Low (USD)': 'low'})

    return df


def rolling_forecast(df: pd.DataFrame, test_ratio: float = 0.3) -> pd.DataFrame:

    prediction_roll = pd.Series(dtype = 'float64')
    test_data = df[ma.floor(len(df.index) * (1 - test_ratio)) + 1:]

    for end_date in test_data.index:
        train_data = df[:end_date - 1]
        # TODO: define knn-regression model
        pred = pd.mean(train_data)
        prediction_roll.loc[end_date] = pred

    return pd.DataFrame({'pred': prediction_roll, 'real': test_data['price']})

<h3>Importing data</h3>

In [4]:
df_ada = get_csv('./data/Cardano.csv')
df_eth = get_csv('./data/ethereum.csv')
df_sol = get_csv('./data/solana.csv')
df_doge = get_csv('./data/dogecoin.csv')